In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install --quiet automl_alex

In [1]:
import automl_alex
import sklearn
import time
from automl_alex import AutoML, AutoMLClassifier
print(automl_alex.__version__)

0.07.18


In [2]:
RANDOM_SEED = 42

# Classifier

## Data

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
dataset = fetch_openml(name='credit-g', version=1, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.25, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((750, 20), (250, 20))

## Model

In [5]:
model = AutoML(X_train, y_train, X_test, type_of_estimator='classifier', random_state=RANDOM_SEED)

# or Simply
model = AutoMLClassifier(X_train, y_train, X_test, random_state=RANDOM_SEED, verbose=1)

Source X_train shape:  (750, 20) | X_test shape:  (250, 20)
##################################################
Auto detect cat features:  13
> Start preprocessing Data
> Generate cat encodet features
 +  55  Features from  OneHotEncoder
 +  44  Features from  HelmertEncoder
 +  54  Features from  HashingEncoder
 +  16  Features from  FrequencyEncoder
> Generate Frequency Encode num features
 +  4  Frequency Encode Num Features 
> Clean Nans in num features
> Generate interaction Num Features
 +  24  Interaction Features
> Normalization Features
##################################################
> Total Features:  201
##################################################
New X_train shape:  (750, 201) | X_test shape:  (250, 201)


In [6]:
%%time
predict_test, predict_train = model.fit_predict(timeout=1000, verbose=2)


 Opt BestModels
One iteration takes ~ 0.8 sec
> Start Auto calibration parameters
> Start optimization with the parameters:
CV_Folds =  10
Score_CV_Folds =  5
Feature_Selection =  True
Opt_lvl =  3
Cold_start =  122.0
Early_stoping =  244.0
Metric =  roc_auc_score
Direction =  maximize
##################################################
Default model OptScore = 0.625
Optimize: : 319it [21:48,  2.29s/it,  | Model: CatBoost | OptScore: 0.7946 | Best roc_auc_score: 0.8262 +- 0.031581]
 EarlyStopping Exceeded: Best Score: 0.7946 roc_auc_score
Optimize: : 319it [21:48,  4.10s/it,  | Model: CatBoost | OptScore: 0.7946 | Best roc_auc_score: 0.8262 +- 0.031581]

 Predict from Models_1
 10%|█         | 1/10 [00:51<07:44, 51.58s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7994 std: 0.0483
 20%|██        | 2/10 [01:12<05:39, 42.47s/it]
 Mean Score roc_auc_score on 30 Folds: 0.8006 std: 0.040451
 30%|███       | 3/10 [01:21<03:45, 32.27s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7973 std: 0.04

In [7]:
predict_test[:5]

array([0.51055295, 0.52282972, 0.62619949, 0.48629442, 0.23862737])

In [10]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predict_test),4))

Test AUC:  0.817


In [11]:
model.stack_models_cfgs

,score_opt,model_score,score_std,model_name,model_param,wrapper_params,cat_encoders,columns,cv_folds
0,0.7946,0.8262,0.031581,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, age, OneHotEncoder_existing_credits...",10
1,0.7930,0.8200,0.026979,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_existing_credits, OneHotEncoder...",10
2,0.7870,0.8199,0.032916,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_existing_credits, OneHotEncoder...",10
3,0.7850,0.8178,0.032841,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_existing_credits, OneHotEncoder...",10
4,0.7850,0.8040,0.019039,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, credit_amount, age, OneHotEncoder_i...",10
5,0.7837,0.8180,0.034271,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_existing_credits, OneHotEncoder...",10
6,0.7831,0.8211,0.037984,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_existing_credits, OneHotEncoder...",10
7,0.7820,0.8222,0.040157,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[num_dependents, OneHotEncoder_installment_com...",10
8,0.7814,0.8098,0.028421,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_existing_credits, OneHotEncoder...",10
9,0.7804,0.7995,0.019051,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_existing_credits, OneHotEncoder...",10
